# Manual for Setting up YOLO with Strong Compute's ISC
This guide was created to help setup YOLO with Strong Compute's ISC for our Capstone unit in Semester 2, 2024. I will also provide a guide on how to access the dataset provided by the client and use it to train a model.  

**Disclaimer:** Some of the following code was appropriated from a template provided by Martin Ong on the #strong-isc channel on Discord.

## Step 1: Mounting the Dataset
Before starting the container on Strong Compute's website, ensure that you have mounted the data so that you can access it. Follow the guide [here](https://strong-compute.gitbook.io/developer-docs/basics/datasets) to see how to mount the data but also how to access the data in your training script.  

**NOTE**: don't worry if you can't see the `/data` folder in your workspace - it's actually located in a different parent directory from the `/root` directory that we are all working in. I'll explain in a later step how we can access it for training and validation.

## Step 2: Setup a new Virtual Env
Before we begin actually writing the code, it's important that we setup a new virtual environment to play around in. 

In [7]:
cd ~
python3 -m virtualenv ~/.test_run # replace 'test_run' with whatever you want to name your new virtual env
source ~/.test_run/bin/activate # again, remember to replace 'test_run' with the name of your virtual env

<class 'SyntaxError'>: invalid syntax (<ipython-input-7-8a7cac5e9316>, line 1)

Once you've activated the virtual environment, you need to download the relevant python packages that we'll be using for our YOLO model. These are:
* Ultralytics; and
* PyTorch.

In [ ]:
pip install ultralytics torch

## Step 3: YOLO Script
Now that you've activated your virtual environment and downloaded the required package, it's time to write our python `train.py` script and the relevant `.isc` script which contains all the information that the ISC needs to run it. 

### Step 3.1 Training Script
Create a new file called `train.py`. This will contain all the code for our training script. Copy and paste the following code, remembering to **replace the commented parts with your own directories**

In [1]:
## train.py
import os
import torch
from ultralytics import YOLO
from torch.utils.data import DataLoader, RandomSampler

def main():
    if os.environ.get('OUTPUT_PATH'):
        output_path = f"{os.environ.get('OUTPUT_PATH')}/log.txt"
        output_file = open(output_path, "w")
    else:
        output_path = None

    os.chdir('/root/seb-test-run') # Replace 'seb-test-run' with whatever your directory is

    path = os.getcwd()

    # Create a directory called 'models' which contains the pre-trained YOLOv8 model
    model_path = os.path.join(path, "models", "yolov8n.pt")

    # Create a directory called 'misc' which contains the test image. This line is optional
    image_path = os.path.join(path, "misc", "test_image.jpg")

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = YOLO(model_path).to(device)
    
    # Modify the model's trainer to use RandomSampler
    trainer = model.trainer
    train_dataset = trainer.train_loader.dataset  # Get the original training dataset

    # Create a new DataLoader with RandomSampler
    train_loader = DataLoader(train_dataset, 
                              sampler=RandomSampler(train_dataset),
                              batch_size=trainer.train_loader.batch_size,
                              num_workers=trainer.train_loader.num_workers)

    # Replace the trainer's train_loader with the new DataLoader
    trainer.train_loader = train_loader

    # Now proceed with training
    results = model.train(data="data.yaml", 
                          epochs=3, 
                          imgsz=1024)

    results = model(image_path)  # predict on an image
    for result in results:
        result.show()  # display to screen


if __name__ == "__main__":
    main()


<class 'ModuleNotFoundError'>: No module named 'torch'

### Step 3.2 Updating the TOML File
A TOML file basically communicates important information about your experiment to the ISC. Create a file called `test-run.isc` and place the following lines inside. It's important that you replace:
* **isc_project_id** with your own project ID. This is found on the Strong Compute dashboard.
* **experiment_name [Optional]** with whatever name you want to call your experiment.
* **dataset_id** with the dataset ID. This is also found on the Strong Compute dashboard.
* **command** with the directories to your training script. i.e., command = "source ~/\<virtual env>/venv/bin/activate && python3 ~/\<path to training script>".

In [6]:
"""
isc_project_id = <Project_Id>
experiment_name = "CUDA test"
gpu_type = "24GB VRAM GPU"
gpus = 1
output_path = "~/outputs/cuda_test"
compute_mode = "cycle"
dataset_id = "<your-dataset-id>"
command = "source ~/test_run/venv/bin/activate && python3 ~/test_run/train.py"
"""

'\nisc_project_id = <Project_Id>\nexperiment_name = "CUDA test"\ngpu_type = "24GB VRAM GPU"\ngpus = 1\noutput_path = "~/outputs/cuda_test"\ncompute_mode = "cycle"\ncommand = "source ~/test_run/venv/bin/activate && python3 ~/test_run/src/train.py"\n'

## Step 4: Accessing the Data
Now that we have the training script set up and the relevant `.isc` file, we need to ensure that our YOLO model can actually access the data for training. As I mentioned earlier, the `/data` folder is located in a different parent directory which we actually can't edit, so we need to create a separate `fmg_data.yaml` file in our own directory which tells the YOLO compiler where to look for the data. This is because the original `fmg_data.yaml` file located inside the `/data/fmg_data` directory tells the YOLO compiler to look in `/home/calvin/fmg_data/` for the training and validation images, but this is incorrect.

To fix this issue, copy the `fmg_data.yaml` file into the same directory as your python training script and replace the `path` variable with `/data/fmg_data` as seen in the below. The reason for this is so that we can tell our python script in step 3.1 which `.yaml` file to access so that it correctly locates the data. **[Optional]** You can also specify in which directory you want YOLO to store the output files.

In [ ]:
# fmg_data.yaml
path: /data/fmg_data
train: train
val: val

# Specify the output directory
output_dir: <desired output path>

names:
  0: Moxy
  1: Haul Truck
  2: Front End Loader
  3: Excavator
  4: Crane
  5: Franna
  6: Scissor Lift
  7: Elevated Work Platform
  8: Drum Roller
  9: Bus
  10: Bobcat
  11: Drill Rig
  12: Cone
  13: Person
  14: Light Vehicle
  15: Water Truck
  16: Grader
  17: Fork Lift


You may also need to update the directory that YOLO looks for the data in. When the ultralytics package is downloaded using pip, it creates a settings file located in `/root/.config/Ultralytics/settings.yaml` which tells the YOLO package where to look for the data. The problem is that by default it tells the YOLO package to look inside a `datasets` folder inside wherever directory it was installed in. This directory doesn't exist for us though, so if you try to run the training script as it is, you might receive an error that looks something like this:

In [7]:
# Error message
Ultralytics YOLOv8.2.81 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24241MiB)
Traceback (most recent call last):
  File "/root/.first_train/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 531, in get_dataset
    data = check_det_dataset(self.args.data)
  File "/root/.first_train/lib/python3.10/site-packages/ultralytics/data/utils.py", line 329, in check_det_dataset
    raise FileNotFoundError(m)
FileNotFoundError: 
Dataset '/data/fmg_data/fmg_data.yaml' images not found ⚠️, missing path '/home/calvin/fmg_data/val'
Note dataset download directory is '/root/seb-test-run/datasets'. You can update this in '/root/.config/Ultralytics/settings.yaml'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/root/seb-test-run/train.py", line 33, in <module>
    main()
  File "/root/seb-test-run/train.py", line 22, in main
    results = model.train(data=data_path, 
  File "/root/.first_train/lib/python3.10/site-packages/ultralytics/engine/model.py", line 807, in train
    self.trainer = (trainer or self._smart_load("trainer"))(overrides=args, _callbacks=self.callbacks)
  File "/root/.first_train/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 134, in __init__
    self.trainset, self.testset = self.get_dataset()
  File "/root/.first_train/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 535, in get_dataset
    raise RuntimeError(emojis(f"Dataset '{clean_url(self.args.data)}' error ❌ {e}")) from e
RuntimeError: Dataset '/data/fmg_data/fmg_data.yaml' error ❌ 
Dataset '/data/fmg_data/fmg_data.yaml' images not found ⚠️, missing path '/home/calvin/fmg_data/val'
Note dataset download directory is '/root/seb-test-run/datasets'. You can update this in '/root/.config/Ultralytics/settings.yaml'

<class 'SyntaxError'>: invalid character '🚀' (U+1F680) (<ipython-input-7-ba1f7fa04d86>, line 1)

To fix it, open the `settings.yaml` located in `/root/.config/Ultralytics/settings.yaml` (if you're using VSCode you can do this by clicking on the search bar up the top and choosing 'go to file' and then copying and pasting this path). The file will look something like:

In [1]:
# settings.yaml
settings_version: 0.0.4
datasets_dir: /root/seb-test-run/datasets
weights_dir: weights
runs_dir: runs
...

<class 'SyntaxError'>: invalid syntax (<ipython-input-1-aa76fbd1c30a>, line 2)

Just replace `/root/seb-test-run/datasets` with the directory to the `fmg_data.yaml` file that you just created.

## Step 5: Run an Experiment
Now that we have set everything up and YOLO knows where to look for the data, we can train the model using YOLO.

Launch your experiment by running the following commands.

In [8]:
cd ~/seb-test-run
isc train test-run.isc

/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


[Errno 44] No such file or directory: '/home/pyodide/seb-test-run'
/drive


You can track the status of your experiment in the terminal by typing:

In [ ]:
isc experiments

## Step 6: Accessing Results
To view the results of each experiment, check the log from the ISC experiment as YOLO will tell you where it stored the results of the training. This should by default be inside a folder named `runs` which contains subdirectories for `train1`, `train2` and so on.